In [25]:
import time
import datetime

import numpy as np
import pandas as pd
import optgbm as opt
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

In [26]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [28]:
from sklearn.preprocessing import LabelEncoder
features_target = ['target'] 

for feature in features_target:
    le = LabelEncoder()
    le.fit(train_df[feature])
    train_df[feature] = le.transform(train_df[feature])

In [29]:
print(len(train_df))
non_id_columns = train_df.columns[train_df.columns != 'id']
train_df = train_df.drop_duplicates(subset=non_id_columns)
print(len(train_df))

200000
199894


In [30]:
target = train_df['target']
train_features = train_df.drop(['target', 'id'], 1)

train_percent = 0.50

X_train, X_test, y_train, y_test = train_test_split(train_features, target, stratify=target, train_size=train_percent)
print('train count: ', len(y_train))
print('test count: ', len(y_test))

train count:  99947
test count:  99947


In [31]:
def do_transform(the_train, the_test, scaler):
    full_df = pd.concat([the_train, the_test])
    scaler.fit(full_df)
    encoder.transform(the_train), encoder.transform(the_test)
        
    return the_train, the_test

In [32]:
def do_train(X_train, y_train):
    print(datetime.datetime.now())
    start = time.time()

    lgbm = opt.LGBMClassifier()
    lgbm.fit(X_train, y_train)

    minutes = (time.time() - start) / 60
    print(round(minutes, 2))
    print(datetime.datetime.now())
    
    return lgbm

In [34]:
def do_test(X_test, y_test, model):
    test_preds = model.predict_proba(X_test)
    print(abs(log_loss(y_test, test_preds)))
    return test_preds

In [35]:
def do_all(X_test, y_test, X_train, y_train, scaler):
    X_train_trainsformed, X_test_trainsformed = do_transform(X_train, X_test, s_scaler)
    model = do_train(X_train_trainsformed, y_train)
    preds = do_test(X_test_trainsformed, y_test, s_model)
    return model, preds

In [61]:
# sklearn.preprocessing.StandardScaler
from sklearn.preprocessing import StandardScaler
s_scaler = StandardScaler()
s_model, s_preds = do_all(X_test, y_test, X_train, y_train, s_scaler)

In [ ]:
# sklearn.preprocessing.RobustScaler
from sklearn.preprocessing import RobustScaler
r_scaler = RobustScaler()
r_model, r_preds = do_all(X_test, y_test, X_train, y_train, r_scaler)

In [ ]:
# sklearn.preprocessing.MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
mm_scaler = MinMaxScaler()
mm_model, mm_preds = do_all(X_test, y_test, X_train, y_train, mm_scaler)

In [ ]:
# sklearn.preprocessing.Normalizer
from sklearn.preprocessing import Normalizer
the_normalizer = Normalizer()
n_model, n_preds = do_all(X_test, y_test, X_train, y_train, the_normalizer)